In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import socket
socket.setdefaulttimeout(1)

import pandas as pd
import schema_mapping
from pathlib import Path
from tqdm import tqdm
    

In [20]:
df = pd.read_csv("./hospital_price_transparency/data/hospitals.csv")
out_path = Path("./hospital_price_transparency/")
df_url_present = df[~df["cdm_url"].isna()]
df_csv = df_url_present[df_url_present["cdm_url"].str.endswith(".csv")]

In [21]:
# infer the schema map for a few example hospital URLs
for ccn, url in tqdm(df_csv[["ccn", "cdm_url"]].values[0:10]):
    # print(ccn, url)
    header = schema_mapping.read_csv_header(url=url, num_rows=3)
    schema_map = schema_mapping.infer_schema_map(header)
    schema_mapping.write_schema_map(schema_map=schema_map, out_file=out_path / "schema_maps" / f"ccn={ccn}.json")

100%|██████████| 10/10 [00:10<00:00,  1.04s/it]


Now try reading the data in according to the standardized schema, using Apache Arrow:

https://arrow.apache.org/docs/python/generated/pyarrow.Table.html


In [22]:
import pyarrow as pa
import json
import pyarrow.parquet as pq
import schema

In [23]:
schema.HOSPITAL_PRICE_TRANSPARENCY_SCHEMA

total_charge: float
description: string
ccn: int32
ein: int32
billing_code_type: string
billing_code: string
negotiated_rate: float
billing_code_modifier: string

In [27]:
for file_path in (out_path / "schema_maps").iterdir():
    print(file_path)
    ccn = file_path.name.split("=")[1].split(".")[0]
    url = df_csv[df_csv["ccn"] == ccn]["cdm_url"].item()
    with file_path.open("r") as f:
        schema_map = json.loads(f.read())
    unique_mapped = list(set(schema_map.values()))
    if "total_charge" in unique_mapped:
        first_matching_column = next(
            key for key, value in schema_map.items() if "total_charge" in value
        )
        try:
            df_ccn = pd.read_csv(url)
            total_charge_list = df_ccn[first_matching_column].head().tolist()
            print(total_charge_list)
            size = len(total_charge_list)
            empty_list = [None] * size
            pylist = [
                {"total_charge": total_charge, "ccn": int(ccn)}
                for total_charge in total_charge_list
            ]
            table = pa.Table.from_pylist(
                pylist,
                schema=schema.HOSPITAL_PRICE_TRANSPARENCY_SCHEMA,
            )
            pq.write_table(table, out_path / "standardized_data" / f"ccn={ccn}.parquet")
        except Exception as e:
            print(e)
            pass


hospital_price_transparency/schema_maps/ccn=010012.json
hospital_price_transparency/schema_maps/ccn=240014.json
[369.0, 121.0, 0.0, 89.0, 36.0]
hospital_price_transparency/schema_maps/ccn=240043.json


Now query the parquet files using duckdb

In [ ]:
import duckdb 

con = duckdb.connect()
con.execute("SELECT * FROM 'hospital_price_transparency/standardized_data/*.parquet'").df()

,total_charge,description,ccn,ein,billing_code_type,billing_code,negotiated_rate,billing_code_modifier
0,369.0,NaN,240014,NaN,NaN,NaN,NaN,NaN
1,121.0,NaN,240014,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,240014,NaN,NaN,NaN,NaN,NaN
3,89.0,NaN,240014,NaN,NaN,NaN,NaN,NaN
4,36.0,NaN,240014,NaN,NaN,NaN,NaN,NaN
